<a href="https://colab.research.google.com/github/janbanot/msc-project/blob/main/experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiment: XAI & Representation Engineering for Toxic Comment Classification

Skonsolidowany notebook eksperymentalny zawierajacy:
- **Modul 0**: Konfiguracja i Przygotowanie
- **Modul A**: Porownanie metod XAI (Comprehensiveness)
- **Modul B**: Analiza warstwowa (RepE)
- **Modul C**: Test stabilnosci (Semantic Robustness)
- **Modul D**: Test skutecznosci sterowania (Steering)

---
## Modul 0: Konfiguracja i Przygotowanie

In [ ]:
!uv pip install transformers datasets captum quantus accelerate bitsandbytes sentence-transformers nltk scipy

In [ ]:
# ===================================================
# 0.1 IMPORTY
# ===================================================

import os
import re
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from datetime import datetime

# Dane i preprocessing
from datasets import Dataset
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Modele
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)

# XAI
from captum.attr import IntegratedGradients, InputXGradient

# Stabilnosc semantyczna (Modul C)
from sentence_transformers import SentenceTransformer
from scipy.stats import spearmanr

# Synonimy (Modul C - alternatywa dla Mistrala)
import nltk
from nltk.corpus import wordnet

# Zarzadzanie pamiecia
import gc

# Wizualizacja w notebooku
from IPython.display import Image, display, Markdown

# Opcje wyswietlania Pandas
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)

In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
# ===================================================
# 0.2 KONFIGURACJA GLOBALNA
# ===================================================

# === Sciezki ===
DATA_PATH = "/drive/MyDrive/msc-project/jigsaw-toxic-comment/train.csv"
MODEL_CHECKPOINT = "/drive/MyDrive/msc-project/models/distilbert-jigsaw-full_20260125_133112"

TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
RESULTS_DIR = f"/drive/MyDrive/msc-project/results_{TIMESTAMP}"

# === Parametry ogolne ===
BATCH_SIZE = 32
MAX_SEQUENCE_LENGTH = 256
CLASSIFICATION_THRESHOLD = 0.5
DF_SIZE = 3000  # Ograniczenie wielkosci zbioru danych

# === Parametry XAI (Modul A) ===
N_SAMPLES_XAI = 100  # Liczba probek dla metod XAI (IG / InputXGradient)
XAI_N_STEPS = 50     # Liczba krokow dla Integrated Gradients
TOP_K_TOKENS = 5     # Liczba najwazniejszych tokenow do analizy Comprehensiveness

# === Parametry RepE (Modul B) ===
N_SAMPLES_PROBE = 1000  # Liczba probek do analizy warstwowej
TARGET_LAYER_INDEX = 5  # Warstwa docelowa (najlepsza separowalnosc liniowa)

# === Parametry stabilnosci (Modul C) ===
N_SAMPLES_STABILITY = 50     # Liczba par tekst-parafraza
PARAPHRASE_MIN_SIMILARITY = 0.7  # Minimalny cosine similarity dla akceptacji parafrazy
PARAPHRASE_SEED = 42        # Seed dla reproducibility
SMOOTHGRAD_N_SAMPLES = 30   # Liczba probek szumu w SmoothGrad
SMOOTHGRAD_NOISE_STD = 0.1  # Odchylenie standardowe szumu gaussowskiego
SYNONYM_REPLACE_RATIO = 0.3 # Procent slow do zamiany na synonimy

# === Parametry Steering (Modul D) ===
STEERING_ALPHA = -3.0       # Sila wektora sterujacego (ujemna = detoksykacja)
ALPHA_VALUES = [-10.0, -20.0, -50.0]  # Wartosci alpha do testowania
N_SAMPLES_PER_CLASS = 25    # Liczba probek na klase do testu steeringu
MISTRAL_MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"  # Model LLM do parafrazowania

# === Urzadzenie obliczeniowe ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Urzadzenie: {device}")

# Tworzenie katalogu wynikow
os.makedirs(RESULTS_DIR, exist_ok=True)
print(f"Wyniki beda zapisane w: {RESULTS_DIR}")

In [ ]:
# ===================================================
# 0.3 PRZYGOTOWANIE DANYCH I MODELU
# ===================================================


def clean_text(example):
    """
    Czysci tekst komentarza, usuwajac niepozadane elementy i normalizujac format.

    Funkcja stosowana zarowno podczas treningu jak i ewaluacji, aby zapewnic
    spojnosc przetwarzania danych.

    Argumenty:
        example: Slownik zawierajacy klucz 'comment_text' z tekstem do oczyszczenia

    Zwraca:
        Zmodyfikowany slownik example z oczyszczonym tekstem w polu 'comment_text'

    Operacje czyszczenia:
        - Konwersja na male litery (wymagane dla modeli BERT typu uncased)
        - Usuniecie linkow URL (http/https/www)
        - Usuniecie adresow IP
        - Usuniecie metadanych Wikipedii (talk pages, timestampy UTC)
        - Normalizacja bialych znakow (spacje, newline, non-breaking space)
        - Usuniecie cudzyslowow z poczatku i konca
    """
    text = example["comment_text"]
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", "", text)
    text = re.sub(r"\(talk\)", "", text)
    text = re.sub(r"\d{2}:\d{2}, \w+ \d{1,2}, \d{4} \(utc\)", "", text)
    text = text.replace("\n", " ").replace("\xa0", " ")
    text = text.strip(' "')
    text = re.sub(r"\s+", " ", text).strip()
    example["comment_text"] = text
    return example


def prepare_environment():
    """
    Przygotowuje srodowisko eksperymentalne: wczytuje dane, tokenizuje i laduje model.

    Zwraca:
        Tuple zawierajacy:
        - model: Wytrenowany model DistilBERT do klasyfikacji toksycznosci
        - tokenizer: Tokenizer dopasowany do modelu
        - eval_dataset: Zbior testowy z przetworzonymi danymi

    Kroki przygotowania:
        1. Wczytanie danych z pliku CSV
        2. Preprocessing tekstow
        3. Ladowanie tokenizera
        4. Tokenizacja tekstow (padding do MAX_SEQUENCE_LENGTH)
        5. Przygotowanie etykiet binary classification
        6. Podzial na zbior treningowy i testowy
        7. Zaladowanie wytrenowanego modelu
    """
    print(">>> [SETUP] Wczytywanie i przetwarzanie danych...")

    # 1. Wczytanie danych
    try:
        df = pd.read_csv(DATA_PATH).head(DF_SIZE)
        dataset = Dataset.from_pandas(df)
    except FileNotFoundError:
        raise FileNotFoundError(
            f"Nie znaleziono pliku: {DATA_PATH}. Sprawdz sciezke w Konfiguracji Globalnej."
        )

    # 2. Preprocessing
    dataset = dataset.map(clean_text)

    # 3. Ladowanie tokenizera zgodnego z modelem
    print(f">>> [SETUP] Ladowanie tokenizera z: {MODEL_CHECKPOINT}...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
    except OSError:
        print(
            f"Blad: Nie znaleziono tokenizera w {MODEL_CHECKPOINT}. Pobieram domyslny 'distilbert-base-uncased'."
        )
        tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    # 4. Tokenizacja
    def tokenize_function(examples):
        """Tokenizuje teksty z paddingiem do stalej dlugosci MAX_SEQUENCE_LENGTH."""
        return tokenizer(
            examples["comment_text"],
            padding="max_length",
            truncation=True,
            max_length=MAX_SEQUENCE_LENGTH,
        )

    tokenized_dataset = dataset.map(tokenize_function, batched=True)

    # 5. Przygotowanie etykiet binary classification
    label_cols = ["toxic"]

    def create_labels(example):
        """Pobiera kolumne 'toxic' i tworzy etykiete."""
        example["labels"] = [float(example[col]) for col in label_cols]
        return example

    final_dataset = tokenized_dataset.map(create_labels)

    # Ustawienie formatu PyTorch (usuniecie kolumn tekstowych, zachowanie tylko tensorow)
    cols_to_keep = ["input_ids", "attention_mask", "labels"]
    final_dataset.set_format("torch", columns=cols_to_keep)

    # 6. Podzial na zbior treningowy i testowy
    splits = final_dataset.train_test_split(test_size=0.2, seed=42)
    eval_dataset = splits["test"]

    # 7. Ladowanie wytrenowanego modelu
    print(f">>> [SETUP] Ladowanie wytrenowanego modelu z: {MODEL_CHECKPOINT}...")
    try:
        model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_CHECKPOINT, num_labels=1, problem_type="single_label_classification"
        )
    except OSError:
        raise OSError(
            f"Nie znaleziono modelu w sciezce: {MODEL_CHECKPOINT}. Upewnij sie, ze najpierw uruchomiles skrypt treningowy."
        )

    # Przelaczenie w tryb ewaluacji (wylacza dropout i batch normalization)
    model.to(device)
    model.eval()

    print(f">>> [SETUP] Srodowisko gotowe. Urzadzenie: {device}")
    print(f">>> [SETUP] Zbior ewaluacyjny: {len(eval_dataset)} probek")
    return model, tokenizer, eval_dataset


# Inicjalizacja srodowiska
model, tokenizer, eval_dataset = prepare_environment()

---
## Czesc 1: Analiza Wiernosci (Fidelity Check)

Cel: Wykazanie, ze metody XAI wskazuja faktycznie wazne cechy.

Metryki:
- **Comprehensiveness**: Czy usuniecie najwazniejszych tokenow zmienia decyzje modelu? (wysoka wartosc = dobra metoda)
- **Sufficiency**: Czy same najwazniejsze tokeny wystarczaja do utrzymania decyzji? (niska wartosc = dobra metoda)

In [ ]:
# ===================================================
# 1. ANALIZA WIERNOSCI (FIDELITY CHECK)
# ===================================================


def experiment_fidelity(model, tokenizer, dataset):
    """
    Porownuje metody XAI (Integrated Gradients vs InputXGradient) pod katem wiernosci wyjasnien.

    Metryki:
        - Comprehensiveness: mierzy spadek pewnosci po usunieciu TOP_K najwazniejszych tokenow.
          Wysoki spadek = metoda XAI dobrze identyfikuje kluczowe cechy.
        - Sufficiency: mierzy spadek pewnosci gdy zachowamy TYLKO TOP_K najwazniejszych tokenow.
          Niski spadek = same najwazniejsze tokeny wystarczaja do utrzymania predykcji.

    Argumenty:
        model: Wytrenowany model klasyfikacyjny DistilBERT
        tokenizer: Tokenizer odpowiadajacy modelowi
        dataset: Zbior danych z etykietami

    Zwraca:
        DataFrame z wynikami porownania metod (comprehensiveness i sufficiency dla IG i IxG)

    Metodologia:
        1. Wybor podzbioru toksycznych przykladow (N_SAMPLES_XAI)
        2. Dla kazdego przykladu:
            a) Obliczenie oryginalnego prawdopodobienstwa toksycznosci
            b) Identyfikacja TOP_K_TOKENS najwazniejszych tokenow (IG i InputXGradient)
            c) Comprehensiveness: maskowanie tych tokenow i ponowna predykcja
            d) Sufficiency: zachowanie TYLKO tych tokenow i ponowna predykcja
        3. Wizualizacja wynikow jako boxplot (2 subploty)
    """
    print("\n>>> [CZESC 1] Uruchamianie analizy wiernosci (IG vs IxG)...")
    model.eval()

    # Filtrowanie tylko toksycznych przykladow (indeks 0 = etykieta 'toxic')
    toxic_indices = [i for i, labels in enumerate(dataset["labels"]) if labels[0] == 1]
    subset_indices = toxic_indices[:N_SAMPLES_XAI]
    subset = dataset.select(subset_indices)
    print(f"    Wybrano {len(subset)} toksycznych probek do analizy.")

    results = []

    # Funkcja pomocnicza dla Captum (zwraca logity na podstawie embeddings)
    def predict_func(inputs_embeds, attention_mask=None):
        """Wrapper predykcji dla biblioteki Captum."""
        return model(inputs_embeds=inputs_embeds, attention_mask=attention_mask).logits

    ig = IntegratedGradients(predict_func)
    ixg = InputXGradient(predict_func)

    for i in tqdm(range(len(subset)), desc="Ewaluacja XAI (Fidelity)"):
        input_ids = subset[i]["input_ids"].unsqueeze(0).to(device)
        attention_mask = subset[i]["attention_mask"].unsqueeze(0).to(device)
        input_embeds = model.distilbert.embeddings(input_ids)

        # Baseline = embedding tokena [PAD] (punkt odniesienia dla IG)
        baseline = model.distilbert.embeddings(
            torch.tensor(
                [tokenizer.pad_token_id] * MAX_SEQUENCE_LENGTH, device=device
            ).unsqueeze(0)
        )

        # 1. Oryginalne prawdopodobienstwo toksycznosci
        with torch.no_grad():
            orig_out = model(inputs_embeds=input_embeds, attention_mask=attention_mask)
            orig_prob = torch.sigmoid(orig_out.logits)[0, 0].item()

        # --- Funkcje pomocnicze do obliczania metryk ---

        def calculate_comprehensiveness(attr_tensor):
            """
            Comprehensiveness: maskuje TOP_K najwazniejszych tokenow (zamiana na [PAD]).
            Wysoki wynik = metoda dobrze identyfikuje wazne tokeny.

            Zwraca:
                Spadek prawdopodobienstwa (orig_prob - new_prob)
            """
            # Suma po wymiarze embeddingow -> waznosc na poziomie tokenow
            attr_sum = attr_tensor.sum(dim=-1).squeeze(0)
            # Znajdz TOP_K najwazniejszych tokenow
            _, top_indices = torch.topk(attr_sum, k=TOP_K_TOKENS)

            # Maskowanie tokenow (zamiana na [PAD])
            masked_ids = input_ids.clone()
            masked_ids[0, top_indices] = tokenizer.pad_token_id

            with torch.no_grad():
                new_out = model(masked_ids, attention_mask=attention_mask)
                new_prob = torch.sigmoid(new_out.logits)[0, 0].item()

            return orig_prob - new_prob

        def calculate_sufficiency(attr_tensor):
            """
            Sufficiency: zachowuje TYLKO TOP_K najwazniejszych tokenow, reszta -> [PAD].
            Niski wynik = same najwazniejsze tokeny wystarczaja do utrzymania predykcji.

            Zwraca:
                Spadek prawdopodobienstwa (orig_prob - new_prob)
            """
            attr_sum = attr_tensor.sum(dim=-1).squeeze(0)
            _, top_indices = torch.topk(attr_sum, k=TOP_K_TOKENS)

            # Zachowaj TYLKO top-K tokenow, reszta -> [PAD]
            masked_ids = torch.full_like(input_ids, tokenizer.pad_token_id)
            masked_ids[0, top_indices] = input_ids[0, top_indices]

            with torch.no_grad():
                new_out = model(masked_ids, attention_mask=attention_mask)
                new_prob = torch.sigmoid(new_out.logits)[0, 0].item()

            return orig_prob - new_prob

        # 2. Metoda Integrated Gradients
        attr_ig, _ = ig.attribute(
            inputs=input_embeds,
            baselines=baseline,
            target=0,
            n_steps=XAI_N_STEPS,
            additional_forward_args=(attention_mask,),
            return_convergence_delta=True,
        )
        comp_ig = calculate_comprehensiveness(attr_ig)
        suff_ig = calculate_sufficiency(attr_ig)

        # 3. Metoda InputXGradient
        attr_ixg = ixg.attribute(
            inputs=input_embeds, target=0, additional_forward_args=(attention_mask,)
        )
        comp_ixg = calculate_comprehensiveness(attr_ixg)
        suff_ixg = calculate_sufficiency(attr_ixg)

        results.append(
            {
                "text_id": i,
                "original_prob": orig_prob,
                "ig_comprehensiveness": comp_ig,
                "ixg_comprehensiveness": comp_ixg,
                "ig_sufficiency": suff_ig,
                "ixg_sufficiency": suff_ixg,
            }
        )

    # Zapis wynikow
    df_res = pd.DataFrame(results)
    df_res.to_csv(f"{RESULTS_DIR}/fidelity_results.csv", index=False)

    # Wizualizacja: 2 subploty (Comprehensiveness | Sufficiency)
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Subplot 1: Comprehensiveness
    sns.boxplot(
        data=df_res[["ig_comprehensiveness", "ixg_comprehensiveness"]],
        ax=axes[0],
    )
    axes[0].set_title(
        f"Comprehensiveness - Usunieto {TOP_K_TOKENS} Najwazniejszych Tokenow"
    )
    axes[0].set_ylabel("Spadek Prawdopodobienstwa")
    axes[0].set_xticklabels(["IG", "IxG"])

    # Subplot 2: Sufficiency
    sns.boxplot(
        data=df_res[["ig_sufficiency", "ixg_sufficiency"]],
        ax=axes[1],
    )
    axes[1].set_title(
        f"Sufficiency - Zachowano TYLKO {TOP_K_TOKENS} Najwazniejszych Tokenow"
    )
    axes[1].set_ylabel("Spadek Prawdopodobienstwa")
    axes[1].set_xticklabels(["IG", "IxG"])

    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/fidelity_boxplot.png", dpi=150, bbox_inches="tight")
    plt.show()

    # Podsumowanie statystyk
    print("\n=== PODSUMOWANIE WIERNOSCI (FIDELITY) ===")
    print(f"Probek: {len(df_res)}")
    print(f"\nComprehensiveness (wyzsza = lepsza metoda):")
    print(f"  IG  - srednia: {df_res['ig_comprehensiveness'].mean():.4f}, mediana: {df_res['ig_comprehensiveness'].median():.4f}")
    print(f"  IxG - srednia: {df_res['ixg_comprehensiveness'].mean():.4f}, mediana: {df_res['ixg_comprehensiveness'].median():.4f}")
    print(f"\nSufficiency (nizsza = lepsza metoda):")
    print(f"  IG  - srednia: {df_res['ig_sufficiency'].mean():.4f}, mediana: {df_res['ig_sufficiency'].median():.4f}")
    print(f"  IxG - srednia: {df_res['ixg_sufficiency'].mean():.4f}, mediana: {df_res['ixg_sufficiency'].median():.4f}")

    print("\nCzesc 1 zakonczona.")
    return df_res

In [ ]:
df_fidelity = experiment_fidelity(model, tokenizer, eval_dataset)

---
## Czesc 2: Analiza Stabilnosci (Robustness & Stability)

Cel: Wykazanie, ze standardowe metody XAI (IG) sa niestabilne przy parafrazach tekstu,
a technika SmoothGrad znaczaco poprawia stabilnosc wyjasnien.

Podejscie:
- **Dwie metody parafrazowania**: Mistral-7B (semantyczna parafraza) + zamiana synonimow (WordNet)
- **Dwie metody XAI**: Standard IG vs SmoothGrad (IG + szum gaussowski)
- **Dwie metryki porownania**: Korelacja Spearmana (na wspolnym slowniku) + Semantic Overlap (top-K slow)

Oczekiwany wynik: SmoothGrad daje wyzsza korelacje miedzy wyjasnieniam dla oryginalow i parafraz.

In [ ]:
# ===================================================
# 2.1 GENEROWANIE PARAFRAZ (MISTRAL + SYNONIMY)
# ===================================================


def generate_paraphrase_mistral(text, llm_model, llm_tokenizer, llm_device):
    """
    Generuje parafraze toksycznego komentarza za pomoca Mistral-7B z few-shot promptingiem.

    Wykorzystuje agresywne czyszczenie regexem, aby usunac typowe wstepy generowane
    przez model (np. 'Here is your paraphrase').

    Argumenty:
        text: Tekst oryginalny do sparafrazowania
        llm_model: Zaladowany model Mistral-7B
        llm_tokenizer: Tokenizer Mistrala
        llm_device: Urzadzenie obliczeniowe

    Zwraca:
        Oczyszczona parafraza jako string
    """
    few_shot_examples = """
    Task: Paraphrase toxic comments. Maintain intent. Output ONLY the text.

    Original: "you are a complete failure and a waste of space"
    Paraphrase: "you're a total loser who doesn't belong here"

    Original: "shut your mouth you stupid prick"
    Paraphrase: "keep your mouth shut you moronic jerk"

    Original: "get out of this group, no one likes you"
    Paraphrase: "leave this community, nobody wants you here"
    """

    prompt = f"<s>[INST] {few_shot_examples}\nNow paraphrase this:\nOriginal: \"{text}\" [/INST]Paraphrase:"

    inputs = llm_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(llm_device)

    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            pad_token_id=llm_tokenizer.eos_token_id,
        )

    gen_text = llm_tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True
    ).strip()

    # Agresywne czyszczenie regexem
    patterns = [
        r"(?i)^here's a paraphrased version.*?:",
        r"(?i)^here is a paraphrase.*?:",
        r"(?i)^sure, here is.*?:",
        r"(?i)^paraphrased text:",
        r"(?i)^hello there,",
        r"(?i)^the paraphrase is:",
        r"(?i)^original:.*?\n",
    ]

    clean = gen_text.split('\n')[0]
    for p in patterns:
        clean = re.sub(p, "", clean).strip()

    return clean.strip().strip('"')


def generate_synonym_paraphrase(text, replace_ratio=SYNONYM_REPLACE_RATIO):
    """
    Generuje parafraze przez losowa zamiane slow na synonimy z WordNet.

    Metoda deterministyczna i szybka - nie wymaga modelu LLM.
    Zamienia tylko rzeczowniki, czasowniki, przymiotniki i przyslowki.

    Argumenty:
        text: Tekst oryginalny
        replace_ratio: Jaki procent slow zamienic (domyslnie SYNONYM_REPLACE_RATIO)

    Zwraca:
        Tekst z zamienionymi slowami na synonimy
    """
    words = text.split()
    n_to_replace = max(1, int(len(words) * replace_ratio))

    # Wybierz losowe indeksy do zamiany
    np.random.seed(None)  # Losowy seed dla roznorodnosci
    candidate_indices = list(range(len(words)))
    np.random.shuffle(candidate_indices)

    replaced = 0
    new_words = words.copy()

    for idx in candidate_indices:
        if replaced >= n_to_replace:
            break

        word = words[idx].lower().strip('.,!?;:')
        if len(word) < 3:  # Pomijaj krotkie slowa
            continue

        # Szukaj synonimow w WordNet
        synsets = wordnet.synsets(word)
        synonyms = set()
        for syn in synsets:
            for lemma in syn.lemmas():
                name = lemma.name().replace('_', ' ')
                if name.lower() != word:
                    synonyms.add(name)

        if synonyms:
            synonym = np.random.choice(list(synonyms))
            # Zachowaj oryginalna interpunkcje
            suffix = ''
            if words[idx][-1] in '.,!?;:':
                suffix = words[idx][-1]
            new_words[idx] = synonym + suffix
            replaced += 1

    return ' '.join(new_words)


def generate_all_paraphrases(model, tokenizer, dataset):
    """
    Generuje parafrazy obiema metodami (Mistral + Synonimy) dla N_SAMPLES_STABILITY
    toksycznych przykladow. Zarzadza pamiecia GPU - laduje i zwalnia Mistrala.

    Argumenty:
        model: Model DistilBERT (potrzebny do walidacji jakosci parafraz)
        tokenizer: Tokenizer DistilBERT
        dataset: Zbior ewaluacyjny

    Zwraca:
        Lista slownikow z parami (oryginal, parafraza_mistral, parafraza_synonym)
    """
    print("\n>>> [CZESC 2] Generowanie parafraz...")

    # Pobieranie NLTK data (jednorazowo)
    nltk.download('wordnet', quiet=True)
    nltk.download('omw-1.4', quiet=True)

    # Filtrowanie toksycznych przykladow
    toxic_indices = [i for i, labels in enumerate(dataset["labels"]) if labels[0] == 1]
    sample_indices = toxic_indices[:N_SAMPLES_STABILITY]
    print(f"    Wybrano {len(sample_indices)} toksycznych probek.")

    # Dekodowanie tekstow z tokenow
    original_texts = [
        tokenizer.decode(dataset[idx]["input_ids"], skip_special_tokens=True)
        for idx in sample_indices
    ]

    # --- FAZA 1: Generowanie parafraz Mistralem ---
    print("\n    [Faza 1/3] Ladowanie Mistral-7B do generowania parafraz...")

    # Tymczasowo przenosimy DistilBERT na CPU, zeby zwolnic VRAM
    model.cpu()
    torch.cuda.empty_cache()

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    mistral_tokenizer = AutoTokenizer.from_pretrained(MISTRAL_MODEL_ID)
    mistral_tokenizer.pad_token = mistral_tokenizer.eos_token
    mistral_tokenizer.padding_side = "left"

    mistral_model = AutoModelForCausalLM.from_pretrained(
        MISTRAL_MODEL_ID,
        quantization_config=bnb_config,
        device_map="auto",
    )
    print("    Mistral zaladowany.")

    mistral_paraphrases = []
    for text in tqdm(original_texts, desc="Generowanie parafraz (Mistral)"):
        try:
            para = generate_paraphrase_mistral(text, mistral_model, mistral_tokenizer, device)
            mistral_paraphrases.append(para)
        except Exception as e:
            print(f"    Blad Mistral: {e}")
            mistral_paraphrases.append(None)

    # Czyszczenie pamieci po Mistralu
    print("\n    [Faza 2/3] Czyszczenie pamieci GPU po Mistralu...")
    del mistral_model
    del mistral_tokenizer
    torch.cuda.empty_cache()
    gc.collect()

    # Przywracamy DistilBERT na GPU
    model.to(device)
    model.eval()
    print("    Pamiec GPU zwolniona. DistilBERT przywrocony na GPU.")

    # --- FAZA 2: Generowanie parafraz synonimami ---
    print("\n    [Faza 3/3] Generowanie parafraz synonimami (WordNet)...")
    synonym_paraphrases = []
    for text in tqdm(original_texts, desc="Generowanie parafraz (Synonimy)"):
        try:
            para = generate_synonym_paraphrase(text)
            synonym_paraphrases.append(para)
        except Exception as e:
            print(f"    Blad Synonimy: {e}")
            synonym_paraphrases.append(None)

    # --- FAZA 3: Walidacja jakosci i zapis ---
    print("\n    Walidacja jakosci parafraz...")
    semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

    paraphrase_data = []
    for i, orig_text in enumerate(original_texts):
        entry = {
            "idx": i,
            "original": orig_text,
            "mistral_para": mistral_paraphrases[i],
            "synonym_para": synonym_paraphrases[i],
            "mistral_valid": False,
            "synonym_valid": False,
        }

        # Walidacja parafrazy Mistral
        if mistral_paraphrases[i] is not None and len(mistral_paraphrases[i].strip()) > 5:
            embs = semantic_model.encode(
                [orig_text, mistral_paraphrases[i]], convert_to_tensor=True
            )
            cos_sim = F.cosine_similarity(embs[0].unsqueeze(0), embs[1].unsqueeze(0)).item()
            entry["mistral_cos_sim"] = cos_sim
            entry["mistral_valid"] = cos_sim >= PARAPHRASE_MIN_SIMILARITY

        # Walidacja parafrazy synonimowej
        if synonym_paraphrases[i] is not None and len(synonym_paraphrases[i].strip()) > 5:
            embs = semantic_model.encode(
                [orig_text, synonym_paraphrases[i]], convert_to_tensor=True
            )
            cos_sim = F.cosine_similarity(embs[0].unsqueeze(0), embs[1].unsqueeze(0)).item()
            entry["synonym_cos_sim"] = cos_sim
            entry["synonym_valid"] = cos_sim >= PARAPHRASE_MIN_SIMILARITY

        paraphrase_data.append(entry)

    # Zapis checkpoint
    df_paraphrases = pd.DataFrame(paraphrase_data)
    df_paraphrases.to_csv(f"{RESULTS_DIR}/paraphrase_data.csv", index=False)

    n_mistral_valid = sum(1 for d in paraphrase_data if d["mistral_valid"])
    n_synonym_valid = sum(1 for d in paraphrase_data if d["synonym_valid"])
    print(f"\n    Mistral: {n_mistral_valid}/{len(paraphrase_data)} parafraz przeszlo walidacje")
    print(f"    Synonimy: {n_synonym_valid}/{len(paraphrase_data)} parafraz przeszlo walidacje")
    print("    Dane parafraz zapisane do CSV.")

    del semantic_model
    gc.collect()

    return paraphrase_data

In [ ]:
paraphrase_data = generate_all_paraphrases(model, tokenizer, eval_dataset)

In [ ]:
# ===================================================
# 2.2 FUNKCJE ATRYBUCJI (STANDARD IG + SMOOTHGRAD)
# ===================================================


def get_word_attributions(text, model, tokenizer):
    """
    Oblicza atrybucje na poziomie slow za pomoca Standard Integrated Gradients.

    Agreguje atrybucje sub-tokenow do calych slow uzywajac mapowania word_ids().
    Zwraca PELNY slownik atrybucji (nie tylko top-K), co umozliwia korelacje Spearmana.

    Argumenty:
        text: Tekst wejsciowy
        model: Model DistilBERT
        tokenizer: Tokenizer DistilBERT

    Zwraca:
        dict: Slownik {slowo: wartosc_atrybucji} z wartosciami bezwzglednymi
    """
    def predict_func(inputs_embeds, attention_mask=None):
        return model(inputs_embeds=inputs_embeds, attention_mask=attention_mask).logits

    ig = IntegratedGradients(predict_func)

    inputs = tokenizer(
        text, return_tensors="pt", truncation=True,
        padding="max_length", max_length=MAX_SEQUENCE_LENGTH,
    ).to(device)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    emb = model.distilbert.embeddings(input_ids)
    baseline = model.distilbert.embeddings(
        torch.tensor(
            [tokenizer.pad_token_id] * MAX_SEQUENCE_LENGTH, device=device
        ).unsqueeze(0)
    )

    attr, _ = ig.attribute(
        emb, baselines=baseline, target=0,
        n_steps=XAI_N_STEPS,
        additional_forward_args=(attention_mask,),
        return_convergence_delta=True,
    )
    # Suma po wymiarze embeddingow + wartosc bezwzgledna -> waznosc tokenu
    attr_sum = attr.sum(dim=-1).squeeze(0).abs()

    # Agregacja sub-tokenow do slow
    encoding = tokenizer(text, truncation=True, max_length=MAX_SEQUENCE_LENGTH)
    word_ids = encoding.word_ids()

    word_attributions = {}
    for i, word_idx in enumerate(word_ids):
        if word_idx is not None:
            start, end = encoding.token_to_chars(i)
            word = text[start:end].lower().strip()
            if word:  # Pomijaj puste stringi
                word_attributions[word] = word_attributions.get(word, 0) + attr_sum[i].item()

    return word_attributions


def get_word_attributions_smoothgrad(text, model, tokenizer,
                                     n_samples=SMOOTHGRAD_N_SAMPLES,
                                     noise_std=SMOOTHGRAD_NOISE_STD):
    """
    Oblicza atrybucje na poziomie slow za pomoca SmoothGrad.

    SmoothGrad dodaje szum gaussowski do embedddingow wejsciowych i usrednia
    atrybucje z wielu prob. Redukuje to szum w gradientach i daje bardziej
    stabilne wyjasnenia.

    Algorytm:
        1. Dla n_samples iteracji:
            a) Dodaj szum N(0, noise_std) do embedddingow
            b) Oblicz atrybucje IG na zaszumionych embeddingach
        2. Usrednij atrybucje ze wszystkich prob
        3. Agreguj sub-tokeny do slow

    Argumenty:
        text: Tekst wejsciowy
        model: Model DistilBERT
        tokenizer: Tokenizer DistilBERT
        n_samples: Liczba prob szumu (domyslnie SMOOTHGRAD_N_SAMPLES=30)
        noise_std: Odchylenie standardowe szumu (domyslnie SMOOTHGRAD_NOISE_STD=0.1)

    Zwraca:
        dict: Slownik {slowo: usredniona_wartosc_atrybucji}
    """
    def predict_func(inputs_embeds, attention_mask=None):
        return model(inputs_embeds=inputs_embeds, attention_mask=attention_mask).logits

    ig = IntegratedGradients(predict_func)

    inputs = tokenizer(
        text, return_tensors="pt", truncation=True,
        padding="max_length", max_length=MAX_SEQUENCE_LENGTH,
    ).to(device)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    emb = model.distilbert.embeddings(input_ids)
    baseline = model.distilbert.embeddings(
        torch.tensor(
            [tokenizer.pad_token_id] * MAX_SEQUENCE_LENGTH, device=device
        ).unsqueeze(0)
    )

    # Akumulacja atrybucji z wielu zaszumionych prob
    accumulated_attr = torch.zeros_like(emb.squeeze(0)[:, 0])  # [seq_len]

    for _ in range(n_samples):
        # Dodaj szum gaussowski do embedddingow
        noise = torch.randn_like(emb) * noise_std
        noisy_emb = emb + noise

        attr, _ = ig.attribute(
            noisy_emb, baselines=baseline, target=0,
            n_steps=XAI_N_STEPS,
            additional_forward_args=(attention_mask,),
            return_convergence_delta=True,
        )
        # Suma po wymiarze embedddingow -> waznosc tokenu
        attr_sum = attr.sum(dim=-1).squeeze(0).abs()
        accumulated_attr += attr_sum

    # Usrednienie
    averaged_attr = accumulated_attr / n_samples

    # Agregacja sub-tokenow do slow
    encoding = tokenizer(text, truncation=True, max_length=MAX_SEQUENCE_LENGTH)
    word_ids = encoding.word_ids()

    word_attributions = {}
    for i, word_idx in enumerate(word_ids):
        if word_idx is not None:
            start, end = encoding.token_to_chars(i)
            word = text[start:end].lower().strip()
            if word:
                word_attributions[word] = word_attributions.get(word, 0) + averaged_attr[i].item()

    return word_attributions


def calculate_spearman_on_shared_vocab(attrs_orig, attrs_para):
    """
    Oblicza korelacje Spearmana miedzy atrybucjami oryginalnego tekstu i parafrazy
    na wspolnym slowniku (unii slow z obu tekstow).

    Slowa nieobecne w jednym z tekstow otrzymuja atrybucje 0.

    Argumenty:
        attrs_orig: dict {slowo: atrybucja} dla tekstu oryginalnego
        attrs_para: dict {slowo: atrybucja} dla parafrazy

    Zwraca:
        float: Wspolczynnik korelacji Spearmana (lub 0.0 jesli zbyt malo danych)
    """
    # Unia slow z obu tekstow
    all_words = set(attrs_orig.keys()) | set(attrs_para.keys())

    if len(all_words) < 3:  # Zbyt malo danych do korelacji
        return 0.0

    # Budowanie wektorow na wspolnym slowniku (brakujace = 0)
    vec_orig = [attrs_orig.get(w, 0.0) for w in all_words]
    vec_para = [attrs_para.get(w, 0.0) for w in all_words]

    # Sprawdzenie czy wektory nie sa stalymi (spearmanr wymaga wariancji)
    if len(set(vec_orig)) < 2 or len(set(vec_para)) < 2:
        return 0.0

    corr, _ = spearmanr(vec_orig, vec_para)
    return corr if not np.isnan(corr) else 0.0


def calculate_semantic_overlap(words_orig, words_para, semantic_model):
    """
    Mierzy semantyczne podobienstwo miedzy dwoma listami najwazniejszych slow.

    Uzywa modelu SentenceTransformer do kodowania slow, nastepnie oblicza
    srednie maksymalne cosine similarity w obu kierunkach.

    Argumenty:
        words_orig: Lista top-K slow z oryginalnego tekstu
        words_para: Lista top-K slow z parafrazy
        semantic_model: Zaladowany model SentenceTransformer

    Zwraca:
        float: Srednie semantyczne podobienstwo (0.0 - 1.0)
    """
    if not words_orig or not words_para:
        return 0.0
    emb_orig = semantic_model.encode(words_orig, convert_to_tensor=True)
    emb_para = semantic_model.encode(words_para, convert_to_tensor=True)
    cos_sim_matrix = F.cosine_similarity(
        emb_orig.unsqueeze(1), emb_para.unsqueeze(0), dim=2
    )
    score = (
        torch.max(cos_sim_matrix, dim=1)[0].mean()
        + torch.max(cos_sim_matrix, dim=0)[0].mean()
    ).item() / 2
    return score

In [ ]:
# ===================================================
# 2.3 PETLA EWALUACJI STABILNOSCI
# ===================================================


def experiment_stability(model, tokenizer, paraphrase_data):
    """
    Porownuje stabilnosc Standard IG vs SmoothGrad na parafrazach (Mistral + Synonimy).

    Dla kazdej pary (oryginal, parafraza) oblicza:
        - Atrybucje Standard IG i SmoothGrad
        - Korelacje Spearmana na wspolnym slowniku
        - Semantic Overlap top-K slow

    Argumenty:
        model: Model DistilBERT
        tokenizer: Tokenizer DistilBERT
        paraphrase_data: Lista slownikow z generate_all_paraphrases()

    Zwraca:
        DataFrame z wynikami stabilnosci dla kazdej pary
    """
    print("\n>>> [CZESC 2] Ewaluacja stabilnosci XAI (Standard IG vs SmoothGrad)...")
    model.eval()

    # Ladowanie modelu semantycznego do Semantic Overlap
    semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

    results = []

    # Przetwarzanie par: Mistral i Synonimy
    para_types = [
        ("mistral", "mistral_para", "mistral_valid"),
        ("synonym", "synonym_para", "synonym_valid"),
    ]

    for para_type_name, para_key, valid_key in para_types:
        print(f"\n    Przetwarzanie parafraz typu: {para_type_name}")

        valid_pairs = [d for d in paraphrase_data if d.get(valid_key, False)]
        print(f"    Liczba poprawnych par: {len(valid_pairs)}")

        for entry in tqdm(valid_pairs, desc=f"Stabilnosc ({para_type_name})"):
            orig_text = entry["original"]
            para_text = entry[para_key]

            try:
                # --- Standard IG ---
                attrs_orig_std = get_word_attributions(orig_text, model, tokenizer)
                attrs_para_std = get_word_attributions(para_text, model, tokenizer)

                # --- SmoothGrad ---
                attrs_orig_smooth = get_word_attributions_smoothgrad(
                    orig_text, model, tokenizer
                )
                attrs_para_smooth = get_word_attributions_smoothgrad(
                    para_text, model, tokenizer
                )

                # --- Korelacja Spearmana (wspolny slownik) ---
                spearman_std = calculate_spearman_on_shared_vocab(
                    attrs_orig_std, attrs_para_std
                )
                spearman_smooth = calculate_spearman_on_shared_vocab(
                    attrs_orig_smooth, attrs_para_smooth
                )

                # --- Semantic Overlap (top-K slow) ---
                top_k_orig_std = sorted(
                    attrs_orig_std.items(), key=lambda x: x[1], reverse=True
                )[:TOP_K_TOKENS]
                top_k_para_std = sorted(
                    attrs_para_std.items(), key=lambda x: x[1], reverse=True
                )[:TOP_K_TOKENS]
                top_k_orig_smooth = sorted(
                    attrs_orig_smooth.items(), key=lambda x: x[1], reverse=True
                )[:TOP_K_TOKENS]
                top_k_para_smooth = sorted(
                    attrs_para_smooth.items(), key=lambda x: x[1], reverse=True
                )[:TOP_K_TOKENS]

                sem_overlap_std = calculate_semantic_overlap(
                    [w for w, _ in top_k_orig_std],
                    [w for w, _ in top_k_para_std],
                    semantic_model,
                )
                sem_overlap_smooth = calculate_semantic_overlap(
                    [w for w, _ in top_k_orig_smooth],
                    [w for w, _ in top_k_para_smooth],
                    semantic_model,
                )

                results.append({
                    "para_type": para_type_name,
                    "idx": entry["idx"],
                    "spearman_standard_ig": spearman_std,
                    "spearman_smoothgrad": spearman_smooth,
                    "sem_overlap_standard_ig": sem_overlap_std,
                    "sem_overlap_smoothgrad": sem_overlap_smooth,
                })

            except Exception as e:
                print(f"    Blad dla idx={entry['idx']}: {e}")
                continue

    # Zapis wynikow
    df_stability = pd.DataFrame(results)
    df_stability.to_csv(f"{RESULTS_DIR}/stability_results.csv", index=False)

    del semantic_model
    gc.collect()

    print(f"\n    Ewaluacja zakonczona. Przetworzono {len(df_stability)} par.")
    return df_stability

In [ ]:
df_stability = experiment_stability(model, tokenizer, paraphrase_data)

In [ ]:
# ===================================================
# 2.4 WIZUALIZACJA I PODSUMOWANIE STABILNOSCI
# ===================================================


def visualize_stability(df_stability):
    """
    Wizualizuje wyniki analizy stabilnosci: porownanie Standard IG vs SmoothGrad
    dla obu typow parafraz (Mistral i Synonimy).

    Tworzy 4 subploty (2x2):
        - Wiersz 1: Korelacja Spearmana (Mistral | Synonimy)
        - Wiersz 2: Semantic Overlap (Mistral | Synonimy)

    Argumenty:
        df_stability: DataFrame z wynikami experiment_stability()

    Zwraca:
        None (wyswietla wykresy i drukuje podsumowanie)
    """
    if df_stability.empty:
        print("Brak danych do wizualizacji.")
        return

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(
        "Analiza Stabilnosci: Standard IG vs SmoothGrad",
        fontsize=16, fontweight="bold",
    )

    para_types = ["mistral", "synonym"]
    para_labels = ["Parafrazy Mistral", "Parafrazy Synonimowe"]

    for col_idx, (ptype, plabel) in enumerate(zip(para_types, para_labels)):
        df_sub = df_stability[df_stability["para_type"] == ptype]

        if df_sub.empty:
            axes[0, col_idx].text(0.5, 0.5, f"Brak danych: {plabel}",
                                   ha='center', va='center', fontsize=12)
            axes[1, col_idx].text(0.5, 0.5, f"Brak danych: {plabel}",
                                   ha='center', va='center', fontsize=12)
            continue

        # Wiersz 1: Korelacja Spearmana
        spearman_data = df_sub[["spearman_standard_ig", "spearman_smoothgrad"]]
        sns.boxplot(data=spearman_data, ax=axes[0, col_idx], palette="Set2")
        axes[0, col_idx].set_title(f"Korelacja Spearmana - {plabel}")
        axes[0, col_idx].set_ylabel("Spearman rho")
        axes[0, col_idx].set_xticklabels(["Standard IG", "SmoothGrad"])
        axes[0, col_idx].axhline(y=0, color='gray', linestyle='--', alpha=0.5)

        # Wiersz 2: Semantic Overlap
        sem_data = df_sub[["sem_overlap_standard_ig", "sem_overlap_smoothgrad"]]
        sns.boxplot(data=sem_data, ax=axes[1, col_idx], palette="Set2")
        axes[1, col_idx].set_title(f"Semantic Overlap (Top-{TOP_K_TOKENS}) - {plabel}")
        axes[1, col_idx].set_ylabel("Semantic Overlap")
        axes[1, col_idx].set_xticklabels(["Standard IG", "SmoothGrad"])

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(f"{RESULTS_DIR}/stability_boxplot.png", dpi=150, bbox_inches="tight")
    plt.show()

    # --- Tabela podsumowujaca ---
    print("\n=== PODSUMOWANIE STABILNOSCI ===")
    for ptype, plabel in zip(para_types, para_labels):
        df_sub = df_stability[df_stability["para_type"] == ptype]
        if df_sub.empty:
            print(f"\n{plabel}: Brak danych.")
            continue

        print(f"\n--- {plabel} (n={len(df_sub)}) ---")
        print(f"\nKorelacja Spearmana:")
        print(f"  Standard IG  - srednia: {df_sub['spearman_standard_ig'].mean():.4f}, "
              f"mediana: {df_sub['spearman_standard_ig'].median():.4f}, "
              f"std: {df_sub['spearman_standard_ig'].std():.4f}")
        print(f"  SmoothGrad   - srednia: {df_sub['spearman_smoothgrad'].mean():.4f}, "
              f"mediana: {df_sub['spearman_smoothgrad'].median():.4f}, "
              f"std: {df_sub['spearman_smoothgrad'].std():.4f}")

        print(f"\nSemantic Overlap (Top-{TOP_K_TOKENS}):")
        print(f"  Standard IG  - srednia: {df_sub['sem_overlap_standard_ig'].mean():.4f}, "
              f"mediana: {df_sub['sem_overlap_standard_ig'].median():.4f}, "
              f"std: {df_sub['sem_overlap_standard_ig'].std():.4f}")
        print(f"  SmoothGrad   - srednia: {df_sub['sem_overlap_smoothgrad'].mean():.4f}, "
              f"mediana: {df_sub['sem_overlap_smoothgrad'].median():.4f}, "
              f"std: {df_sub['sem_overlap_smoothgrad'].std():.4f}")

        # Poprawa SmoothGrad vs Standard IG
        spearman_improvement = (
            df_sub['spearman_smoothgrad'].mean() - df_sub['spearman_standard_ig'].mean()
        )
        sem_improvement = (
            df_sub['sem_overlap_smoothgrad'].mean() - df_sub['sem_overlap_standard_ig'].mean()
        )
        print(f"\n  Poprawa SmoothGrad vs Standard IG:")
        print(f"    Spearman:         +{spearman_improvement:.4f}")
        print(f"    Semantic Overlap: +{sem_improvement:.4f}")

    print("\nCzesc 2 zakonczona.")

In [ ]:
visualize_stability(df_stability)

---
## Czesc 3: Inzynieria Reprezentacji i Sterowanie (Utility & Steering)

Cel: Wykazanie uzytecznosci praktycznej - mozemy naprawic model bez retrenowania.

Podejscie:
- **Ekstrakcja aktywacji** z warstwy TARGET_LAYER_INDEX dla zbioru toksycznego i bezpiecznego
- **Wektor sterujacy**: Obliczenie kierunku toksycznosci metoda Difference of Means + normalizacja L2
- **SteeringHook**: PyTorch hook modyfikujacy hidden states w czasie inferencji
- **Test detoksykacji**: Ocena skutecznosci dla roznych wartosci alpha (sila interwencji)

Metryki:
- **Success Rate**: Procent toksycznych przykladow sklasyfikowanych jako bezpieczne po steeringu (cel: >80%)
- **Side Effect Rate**: Procent bezpiecznych przykladow blednie sklasyfikowanych jako toksyczne (cel: <5%)
- **Avg Delta**: Srednia zmiana prawdopodobienstwa toksycznosci

In [26]:
# ===================================================
# 3. INZYNIERIA REPREZENTACJI I STEROWANIE (STEERING)
# ===================================================


class SteeringHook:
    """
    PyTorch hook modyfikujacy hidden states poprzez dodanie wektora sterujacego.

    Podczas forward pass, hook dodaje przeskalowany wektor kierunkowy do aktywacji
    wybranej warstwy transformera. Ujemny wspolczynnik (coeff) przesuwa aktywacje
    w kierunku przeciwnym do toksycznosci (detoksykacja).

    Argumenty:
        vector: Tensor wektora sterujacego (768-dim, znormalizowany L2)
        coeff: Wspolczynnik skalowania (alpha). Ujemny = detoksykacja.
    """

    def __init__(self, vector, coeff):
        self.vector = vector
        self.coeff = coeff

    def __call__(self, module, inputs, output):
        is_tuple = isinstance(output, tuple)
        hidden_states = output[0] if is_tuple else output
        steering_vector = self.vector.to(hidden_states.device, dtype=hidden_states.dtype)
        modified_hidden = hidden_states + (self.coeff * steering_vector)

        if is_tuple:
            return (modified_hidden,) + output[1:]
        else:
            return modified_hidden

def experiment_steering(model, tokenizer, dataset):
    """
    Oblicza wektor sterujacy i testuje skutecznosc detoksykacji dla roznych wartosci alpha.

    Algorytm:
        1. Podzial danych: 800 probek do wyznaczenia wektora, reszta do testow (brak wycieku)
        2. Ekstrakcja aktywacji tokenu [CLS] z warstwy TARGET_LAYER_INDEX
        3. Obliczenie wektora sterujacego: mean(toxic) - mean(safe), normalizacja L2
        4. Dla kazdego alpha z ALPHA_VALUES:
            a) Inteligentny wybor probek (ranking pewnosci modelu)
            b) Pomiar prawdopodobienstwa PRZED i PO steeringu
            c) Obliczenie Success Rate i Side Effect Rate

    Argumenty:
        model: Wytrenowany model DistilBERT
        tokenizer: Tokenizer odpowiadajacy modelowi
        dataset: Zbior ewaluacyjny z etykietami

    Zwraca:
        Tuple (df_summary, df_details):
        - df_summary: DataFrame z podsumowaniem per alpha (success_rate, side_effect_rate, avg_delta)
        - df_details: DataFrame ze szczegolowymi wynikami per probka
    """
    print("\n>>> [CZESC 3] Uruchamianie analizy steeringu...")
    model.eval()

    # -----------------------------------------------
    # 3.1 PODZIAL DANYCH (wektor vs test)
    # -----------------------------------------------
    # Zamiast sztywnego 800, bierzemy 60% dostępnych danych na wektor
    total_len = len(dataset)
    N_FOR_VECTOR = int(total_len * 0.6)

    # Zabezpieczenie: upewnijmy się, że zostaje min. 50 próbek na testy
    if total_len - N_FOR_VECTOR < 50:
        print(f"UWAGA: Bardzo mały zbiór danych ({total_len}). Wyniki mogą być niestabilne.")

    vector_subset = dataset.select(range(N_FOR_VECTOR))
    test_subset = dataset.select(range(N_FOR_VECTOR, total_len))

    print(f"    Całkowity rozmiar zbioru eval: {total_len}")
    print(f"    Próbek do wyznaczenia wektora: {len(vector_subset)}")
    print(f"    Próbek do testowania efektów:  {len(test_subset)}")

    # -----------------------------------------------
    # 3.2 EKSTRAKCJA AKTYWACJI
    # -----------------------------------------------
    print(f"    Ekstrakcja aktywacji z warstwy {TARGET_LAYER_INDEX}...")
    layers_data = {TARGET_LAYER_INDEX: []}
    all_labels = []

    loader = torch.utils.data.DataLoader(vector_subset, batch_size=BATCH_SIZE)

    for batch in tqdm(loader, desc="Ekstrakcja aktywacji"):
        input_ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        labels = batch["labels"][:, 0].cpu().numpy()

        with torch.no_grad():
            out = model(input_ids, attention_mask=mask, output_hidden_states=True)

        all_labels.extend(labels)
        # Token [CLS] (indeks 0) z wybranej warstwy
        layers_data[TARGET_LAYER_INDEX].append(
            out.hidden_states[TARGET_LAYER_INDEX][:, 0, :].cpu().numpy()
        )

    if len(layers_data[TARGET_LAYER_INDEX]) == 0:
        raise ValueError("Brak danych do ekstrakcji aktywacji. Sprawdź loader.")

    X = np.concatenate(layers_data[TARGET_LAYER_INDEX], axis=0)
    y = np.array(all_labels)
    y_bin = (y > CLASSIFICATION_THRESHOLD).astype(int)

    print(f"    Statystyki probek - Toksyczne: {np.sum(y_bin == 1)}, Bezpieczne: {np.sum(y_bin == 0)}")
    print(f"    Ksztalt aktywacji: {X.shape}")

    # -----------------------------------------------
    # 3.3 OBLICZENIE WEKTORA STERUJACEGO
    # -----------------------------------------------
    # Zabezpieczenie przed brakiem klasy
    if np.sum(y_bin == 1) == 0 or np.sum(y_bin == 0) == 0:
        raise ValueError("W podzbiorze wektorowym brakuje jednej z klas (same toksyczne lub same bezpieczne). Zwiększ N_FOR_VECTOR lub DF_SIZE.")

    print("    Obliczanie wektora sterujacego (Difference of Means)...")
    mean_toxic = np.mean(X[y_bin == 1], axis=0)
    mean_safe = np.mean(X[y_bin == 0], axis=0)
    direction = mean_toxic - mean_safe

    print(f"    Wektor przed normalizacja - L2 Norm: {np.linalg.norm(direction):.6f}")

    # Normalizacja L2
    direction_normed = direction / np.linalg.norm(direction)
    steering_tensor = torch.tensor(direction_normed, dtype=torch.float32).to(device)

    # Zapis wektora
    np.save(f"{RESULTS_DIR}/steering_vector.npy", direction_normed)
    print(f"    Wektor znormalizowany (L2 Norm: {np.linalg.norm(direction_normed):.2f}). Zapisano do: {RESULTS_DIR}/steering_vector.npy")

    # -----------------------------------------------
    # 3.4 FUNKCJA TESTOWANIA STEERINGU
    # -----------------------------------------------

    def test_steering_for_alpha(alpha_value):
        """
        Testuje skutecznosc steeringu dla danej wartosci alpha.
        """
        print(f"\n    --- Testowanie alpha = {alpha_value} ---")

        # Inteligentny wybor probek (ranking pewnosci)
        all_scores = []
        # Ograniczamy zakres przeszukiwania do dostępnej liczby próbek testowych
        search_range = min(300, len(test_subset))

        if search_range == 0:
             raise ValueError("Brak danych testowych (test_subset jest pusty).")

        for i in range(search_range):
            input_ids = test_subset[i]["input_ids"].unsqueeze(0).to(device)
            mask = test_subset[i]["attention_mask"].unsqueeze(0).to(device)
            label = test_subset[i]["labels"][0].item()

            with torch.no_grad():
                logits = model(input_ids, attention_mask=mask).logits
                prob = torch.sigmoid(logits)[0, 0].item()

            all_scores.append({"idx": i, "prob": prob, "label": label})

        # Toksyczne: najwyzsze P(toxic) | Bezpieczne: najnizsze P(toxic)
        toxic_candidates = [x for x in all_scores if x["label"] == 1]

        # Zabezpieczenie jeśli jest mniej kandydatów niż N_SAMPLES_PER_CLASS
        n_samples_actual = min(N_SAMPLES_PER_CLASS, len(toxic_candidates))

        toxic_indices = [
            x["idx"]
            for x in sorted(toxic_candidates, key=lambda x: x["prob"], reverse=True)[
                :n_samples_actual
            ]
        ]

        safe_candidates = [x for x in all_scores if x["label"] == 0]
        n_samples_safe_actual = min(N_SAMPLES_PER_CLASS, len(safe_candidates))

        safe_indices = [
            x["idx"]
            for x in sorted(safe_candidates, key=lambda x: x["prob"])[:n_samples_safe_actual]
        ]

        print(f"    Wybrano: {len(toxic_indices)} toksycznych, {len(safe_indices)} bezpiecznych")

        layer_module = model.distilbert.transformer.layer[TARGET_LAYER_INDEX]
        results = []
        toxic_deltas, safe_deltas = [], []
        success_count, side_effect_count = 0, 0

        def run_inference(indices, label_type):
            nonlocal success_count, side_effect_count
            if not indices:
                return

            for idx in indices:
                input_ids = test_subset[idx]["input_ids"].unsqueeze(0).to(device)
                mask = test_subset[idx]["attention_mask"].unsqueeze(0).to(device)

                # Pomiar PRZED steeringiem
                with torch.no_grad():
                    prob_before = torch.sigmoid(
                        model(input_ids, mask).logits
                    )[0, 0].item()

                # Pomiar PO steeringu (z hookiem)
                hook = SteeringHook(steering_tensor, alpha_value)
                handle = layer_module.register_forward_hook(hook)
                with torch.no_grad():
                    prob_after = torch.sigmoid(
                        model(input_ids, mask).logits
                    )[0, 0].item()
                handle.remove()

                delta = prob_after - prob_before

                if label_type == "TOXIC":
                    success = prob_after < CLASSIFICATION_THRESHOLD
                    status = "SUCCESS" if success else "FAILED"
                    if success:
                        success_count += 1
                    toxic_deltas.append(delta)
                else:
                    side_effect = prob_after > CLASSIFICATION_THRESHOLD
                    status = "SIDE-EFFECT" if side_effect else "OK"
                    if side_effect:
                        side_effect_count += 1
                    safe_deltas.append(delta)

                text = tokenizer.decode(
                    test_subset[idx]["input_ids"], skip_special_tokens=True
                )
                results.append({
                    "label": label_type,
                    "alpha": alpha_value,
                    "prob_before": prob_before,
                    "prob_after": prob_after,
                    "delta": delta,
                    "status": status,
                    "text": text[:100],
                })

        run_inference(toxic_indices, "TOXIC")
        run_inference(safe_indices, "SAFE")

        # Obliczanie średnich (zabezpieczenie przed dzieleniem przez zero)
        success_rate = (success_count / len(toxic_indices) * 100) if toxic_indices else 0
        side_rate = (side_effect_count / len(safe_indices) * 100) if safe_indices else 0
        avg_d_tox = np.mean(toxic_deltas) if toxic_deltas else 0
        avg_d_safe = np.mean(safe_deltas) if safe_deltas else 0

        print(f"    Success Rate: {success_rate:.1f}% | Side Effects: {side_rate:.1f}% | "
              f"Avg Delta Toxic: {avg_d_tox:+.4f} | Avg Delta Safe: {avg_d_safe:+.4f}")

        return results, success_rate, side_rate, avg_d_tox, avg_d_safe

    # -----------------------------------------------
    # 3.5 GLOWNA PETLA - TESTOWANIE DLA ROZNYCH ALPHA
    # -----------------------------------------------
    print(f"\n    Testowanie steeringu dla alpha: {ALPHA_VALUES}")

    all_details = []
    summary_data = []

    for alpha in ALPHA_VALUES:
        results, success_rate, side_effect_rate, avg_delta_toxic, avg_delta_safe = (
            test_steering_for_alpha(alpha)
        )
        all_details.extend(results)
        summary_data.append({
            "alpha": alpha,
            "success_rate": success_rate,
            "side_effect_rate": side_effect_rate,
            "avg_delta_toxic": avg_delta_toxic,
            "avg_delta_safe": avg_delta_safe,
        })

    # Zapis wynikow
    df_summary = pd.DataFrame(summary_data)
    df_details = pd.DataFrame(all_details)
    df_summary.to_csv(f"{RESULTS_DIR}/steering_summary.csv", index=False)
    df_details.to_csv(f"{RESULTS_DIR}/steering_details.csv", index=False)

    print(f"\n    Wyniki zapisane do: {RESULTS_DIR}/steering_summary.csv")
    print(f"    Szczegoly zapisane do: {RESULTS_DIR}/steering_details.csv")

    return df_summary, df_details

In [ ]:
df_steering, df_steering_details = experiment_steering(model, tokenizer, eval_dataset)

In [ ]:
# ===================================================
# 3.6 WIZUALIZACJA I PODSUMOWANIE STEERINGU
# ===================================================


def visualize_steering(df_summary, df_details):
    """
    Wizualizuje wyniki steeringu: tabela podsumowujaca, analiza optymalnosci
    i szczegolowe przyklady przed/po.

    Tworzy:
        - Wykres slupkowy: Success Rate vs Side Effect Rate per alpha
        - Tabele podsumowujaca z ocena optymalnosci
        - Przyklady detoksykacji (toksyczne z najwieksza zmiana)
        - Przyklady efektow ubocznych (bezpieczne blednie sklasyfikowane)
        - Porownanie jednego tekstu dla roznych alpha

    Argumenty:
        df_summary: DataFrame z podsumowaniem per alpha
        df_details: DataFrame ze szczegolowymi wynikami per probka

    Zwraca:
        None (wyswietla wykresy, tabele i przyklady)
    """
    if df_summary.empty:
        print("Brak danych do wizualizacji.")
        return

    # --- Wykres slupkowy ---
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle(
        "Skutecznosc Sterowania (Steering) dla Roznych Wartosci Alpha",
        fontsize=14, fontweight="bold",
    )

    x = np.arange(len(df_summary))
    alpha_labels = [f"alpha={a}" for a in df_summary["alpha"]]

    # Subplot 1: Success Rate i Side Effect Rate
    width = 0.35
    bars1 = axes[0].bar(x - width / 2, df_summary["success_rate"], width, label="Success Rate", color="#2ecc71")
    bars2 = axes[0].bar(x + width / 2, df_summary["side_effect_rate"], width, label="Side Effect Rate", color="#e74c3c")
    axes[0].set_ylabel("Procent (%)")
    axes[0].set_title("Success Rate vs Side Effect Rate")
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(alpha_labels)
    axes[0].legend()
    axes[0].axhline(y=80, color="green", linestyle="--", alpha=0.5, label="Cel Success (80%)")
    axes[0].axhline(y=5, color="red", linestyle="--", alpha=0.5, label="Limit Side Effects (5%)")
    axes[0].set_ylim(0, 105)

    # Etykiety na slupkach
    for bar in bars1:
        h = bar.get_height()
        axes[0].annotate(f"{h:.1f}%", xy=(bar.get_x() + bar.get_width() / 2, h),
                         xytext=(0, 3), textcoords="offset points", ha="center", fontsize=9)
    for bar in bars2:
        h = bar.get_height()
        axes[0].annotate(f"{h:.1f}%", xy=(bar.get_x() + bar.get_width() / 2, h),
                         xytext=(0, 3), textcoords="offset points", ha="center", fontsize=9)

    # Subplot 2: Srednia delta (zmiana prawdopodobienstwa)
    bars3 = axes[1].bar(x - width / 2, df_summary["avg_delta_toxic"], width, label="Avg Delta (Toxic)", color="#3498db")
    bars4 = axes[1].bar(x + width / 2, df_summary["avg_delta_safe"], width, label="Avg Delta (Safe)", color="#f39c12")
    axes[1].set_ylabel("Srednia Zmiana Prawdopodobienstwa")
    axes[1].set_title("Srednia Delta P(toxic)")
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(alpha_labels)
    axes[1].legend()
    axes[1].axhline(y=0, color="gray", linestyle="-", alpha=0.3)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig(f"{RESULTS_DIR}/steering_barplot.png", dpi=150, bbox_inches="tight")
    plt.show()

    # --- Tabela podsumowujaca z ocena ---
    print("\n=== PODSUMOWANIE STEERINGU ===")
    print(df_summary.to_string(index=False))

    print("\n=== ANALIZA OPTYMALNOSCI ===")
    for _, row in df_summary.iterrows():
        alpha = row["alpha"]
        if row["success_rate"] > 80 and row["side_effect_rate"] < 5:
            status = "OPTYMALNE"
        elif row["success_rate"] > 80:
            status = "DOBRA SKUTECZNOSC, WYSOKIE SIDE EFFECTS"
        elif row["side_effect_rate"] < 5:
            status = "NISKIE SIDE EFFECTS, SLABA SKUTECZNOSC"
        else:
            status = "WYMAGA DOSTROJENIA"

        print(f"\n  Alpha = {alpha}: {status}")
        print(f"    Success Rate:  {row['success_rate']:.2f}% (cel: >80%)")
        print(f"    Side Effects:  {row['side_effect_rate']:.2f}% (cel: <5%)")
        print(f"    Avg Delta Toxic: {row['avg_delta_toxic']:+.4f}")
        print(f"    Avg Delta Safe:  {row['avg_delta_safe']:+.4f}")

    # Rekomendacja najlepszego alpha
    optimal_rows = df_summary[
        (df_summary["success_rate"] > 80) & (df_summary["side_effect_rate"] < 5)
    ]
    if len(optimal_rows) > 0:
        best_idx = optimal_rows["success_rate"].idxmax()
        best_alpha = df_summary.loc[best_idx, "alpha"]
        print(f"\n  >>> REKOMENDACJA: Najlepsza wartosc alpha = {best_alpha}")
    else:
        print(f"\n  >>> UWAGA: Zadna wartosc alpha nie spelnia kryteriow optymalnosci.")

    # --- Szczegolowe przyklady ---
    if not df_details.empty:
        # Przyklad detoksykacji (toksyczne z najwieksza negatywna delta)
        print("\n=== PRZYKLADY DETOKSYKACJI (TOXIC -> SAFE) ===")
        best_alpha_for_examples = df_summary.loc[df_summary["success_rate"].idxmax(), "alpha"]
        toxic_examples = df_details[
            (df_details["label"] == "TOXIC") & (df_details["alpha"] == best_alpha_for_examples)
        ].nsmallest(5, "delta")

        for _, row in toxic_examples.iterrows():
            print(f"\n  Tekst: {row['text'][:120]}")
            print(f"  P(toxic) przed: {row['prob_before']:.4f} -> po: {row['prob_after']:.4f} "
                  f"(delta: {row['delta']:+.4f}) [{row['status']}]")

        # Przyklady efektow ubocznych
        print(f"\n=== PRZYKLADY EFEKTOW UBOCZNYCH (SAFE -> TOXIC) ===")
        side_effects = df_details[
            (df_details["label"] == "SAFE") & (df_details["status"] == "SIDE-EFFECT")
        ]

        if len(side_effects) > 0:
            for _, row in side_effects.head(5).iterrows():
                print(f"\n  [alpha={row['alpha']}] Tekst: {row['text'][:120]}")
                print(f"  P(toxic) przed: {row['prob_before']:.4f} -> po: {row['prob_after']:.4f} "
                      f"(delta: {row['delta']:+.4f}) [SIDE-EFFECT]")
        else:
            print("  Brak efektow ubocznych dla zadnej wartosci alpha.")

        # Porownanie jednego tekstu dla roznych alpha
        print(f"\n=== POROWNANIE JEDNEGO TEKSTU DLA ROZNYCH ALPHA ===")
        toxic_texts = df_details[df_details["label"] == "TOXIC"]
        if len(toxic_texts) > 0:
            sample_text = toxic_texts["text"].iloc[0]
            text_comparison = df_details[df_details["text"] == sample_text][
                ["alpha", "prob_before", "prob_after", "delta", "status"]
            ]
            print(f"\n  Tekst: {sample_text[:120]}")
            print(f"  P(toxic) przed steering: {text_comparison['prob_before'].iloc[0]:.4f}")
            print(f"  Po steering:")
            for _, row in text_comparison.iterrows():
                print(f"    alpha={row['alpha']:+.1f}: P(toxic)={row['prob_after']:.4f} "
                      f"(delta: {row['delta']:+.4f}) [{row['status']}]")

    print("\nCzesc 3 zakonczona.")

In [ ]:
visualize_steering(df_steering, df_steering_details)

---
## Czesc 4: Podsumowanie i Wizualizacja Finalna

Sekcja zbierajaca wyniki ze wszystkich trzech modulow eksperymentalnych.
Generuje finalne wykresy do pracy magisterskiej w stylu akademickim.

Wejscia:
- `df_fidelity` (Czesc 1) - Comprehensiveness i Sufficiency dla IG vs IxG
- `df_stability` (Czesc 2) - Korelacja Spearmana i Semantic Overlap dla Standard IG vs SmoothGrad
- `df_steering`, `df_steering_details` (Czesc 3) - Success Rate i Side Effects per alpha

Wyjscia:
- Zbiorcza tabela metryk (`summary_all_metrics.csv`)
- Wykres zbiorczy 3-w-1 (`summary_overview.png`)
- Osobne wykresy per modul (`fig_fidelity.png`, `fig_stability.png`, `fig_steering.png`)
- Automatyczne wnioski tekstowe

In [ ]:
# ===================================================
# 4. PODSUMOWANIE I WIZUALIZACJA FINALNA
# ===================================================


def _setup_academic_style():
    """
    Konfiguruje styl akademicki matplotlib dla wykresow do pracy magisterskiej.

    Ustawia styl 'seaborn-v0_8-paper' (z fallbackiem), zwiekszone fonty,
    rozdzielczosc 300 DPI i palete przyjazna daltonistom.

    Zwraca:
        dict: Paleta kolorow z kluczami 'blue', 'red', 'orange', 'light_blue'
    """
    try:
        plt.style.use('seaborn-v0_8-paper')
    except OSError:
        try:
            plt.style.use('seaborn-paper')
        except OSError:
            pass  # Uzyj domyslnego stylu

    plt.rcParams.update({
        'font.size': 11,
        'axes.labelsize': 12,
        'axes.titlesize': 13,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'legend.fontsize': 10,
        'figure.dpi': 300,
        'savefig.dpi': 300,
        'savefig.bbox': 'tight',
    })

    # Paleta przyjazna daltonistom (ColorBrewer)
    return {
        'blue': '#2c7bb6',
        'red': '#d7191c',
        'orange': '#fdae61',
        'light_blue': '#abd9e9',
    }


def _build_metrics_table(df_fidelity, df_stability, df_steering):
    """
    Buduje zbiorcza tabele metryk ze wszystkich modulow eksperymentalnych.

    Argumenty:
        df_fidelity: DataFrame z Czesci 1 (kolumny: ig/ixg_comprehensiveness/sufficiency)
        df_stability: DataFrame z Czesci 2 (kolumny: para_type, spearman_*, sem_overlap_*)
        df_steering: DataFrame z Czesci 3 (kolumny: alpha, success_rate, side_effect_rate, avg_delta_*)

    Zwraca:
        DataFrame z kolumnami: Module, Metric, Value
    """
    rows = []

    # --- Czesc 1: Fidelity ---
    for method, prefix in [("IG", "ig"), ("IxG", "ixg")]:
        for metric_name, col_suffix in [("Comprehensiveness", "comprehensiveness"), ("Sufficiency", "sufficiency")]:
            col = f"{prefix}_{col_suffix}"
            mean_val = df_fidelity[col].mean()
            std_val = df_fidelity[col].std()
            rows.append({
                "Module": "Fidelity",
                "Metric": f"{method} {metric_name}",
                "Value": f"{mean_val:.4f} +/- {std_val:.4f}",
                "Mean": mean_val,
                "Std": std_val,
            })

    # --- Czesc 2: Stability ---
    for para_type, para_label in [("mistral", "Mistral"), ("synonym", "Synonym")]:
        df_sub = df_stability[df_stability["para_type"] == para_type]
        if df_sub.empty:
            continue
        for method, col in [("Std IG", "spearman_standard_ig"), ("SmoothGrad", "spearman_smoothgrad")]:
            mean_val = df_sub[col].mean()
            std_val = df_sub[col].std()
            rows.append({
                "Module": "Stability",
                "Metric": f"Spearman {method} ({para_label})",
                "Value": f"{mean_val:.4f} +/- {std_val:.4f}",
                "Mean": mean_val,
                "Std": std_val,
            })
        for method, col in [("Std IG", "sem_overlap_standard_ig"), ("SmoothGrad", "sem_overlap_smoothgrad")]:
            mean_val = df_sub[col].mean()
            std_val = df_sub[col].std()
            rows.append({
                "Module": "Stability",
                "Metric": f"Sem. Overlap {method} ({para_label})",
                "Value": f"{mean_val:.4f} +/- {std_val:.4f}",
                "Mean": mean_val,
                "Std": std_val,
            })

    # --- Czesc 3: Steering ---
    best_idx = df_steering["success_rate"].idxmax()
    best = df_steering.loc[best_idx]
    rows.append({"Module": "Steering", "Metric": "Best Alpha", "Value": f"{best['alpha']:.1f}", "Mean": best['alpha'], "Std": 0.0})
    rows.append({"Module": "Steering", "Metric": "Success Rate (best)", "Value": f"{best['success_rate']:.2f}%", "Mean": best['success_rate'], "Std": 0.0})
    rows.append({"Module": "Steering", "Metric": "Side Effect Rate (best)", "Value": f"{best['side_effect_rate']:.2f}%", "Mean": best['side_effect_rate'], "Std": 0.0})
    rows.append({"Module": "Steering", "Metric": "Avg Delta Toxic (best)", "Value": f"{best['avg_delta_toxic']:+.4f}", "Mean": best['avg_delta_toxic'], "Std": 0.0})
    rows.append({"Module": "Steering", "Metric": "Avg Delta Safe (best)", "Value": f"{best['avg_delta_safe']:+.4f}", "Mean": best['avg_delta_safe'], "Std": 0.0})

    return pd.DataFrame(rows)


def _plot_fidelity(ax, df_fidelity, colors):
    """
    Rysuje wykres Fidelity: Comprehensiveness i Sufficiency dla IG vs IxG.

    Argumenty:
        ax: Matplotlib axes do rysowania
        df_fidelity: DataFrame z Czesci 1
        colors: dict z paleta kolorow
    """
    metrics = ['Comprehensiveness', 'Sufficiency']
    ig_means = [
        df_fidelity['ig_comprehensiveness'].mean(),
        df_fidelity['ig_sufficiency'].mean(),
    ]
    ig_stds = [
        df_fidelity['ig_comprehensiveness'].std(),
        df_fidelity['ig_sufficiency'].std(),
    ]
    ixg_means = [
        df_fidelity['ixg_comprehensiveness'].mean(),
        df_fidelity['ixg_sufficiency'].mean(),
    ]
    ixg_stds = [
        df_fidelity['ixg_comprehensiveness'].std(),
        df_fidelity['ixg_sufficiency'].std(),
    ]

    x = np.arange(len(metrics))
    width = 0.35

    ax.bar(x - width / 2, ig_means, width, yerr=ig_stds, label='IG',
           color=colors['blue'], capsize=4, error_kw={'linewidth': 1})
    ax.bar(x + width / 2, ixg_means, width, yerr=ixg_stds, label='IxG',
           color=colors['orange'], capsize=4, error_kw={'linewidth': 1})

    ax.set_ylabel('Spadek prawdopodobienstwa')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics)
    ax.legend()
    ax.set_title('(a) Fidelity: IG vs IxG')
    ax.axhline(y=0, color='gray', linestyle='-', alpha=0.3)


def _plot_stability(ax, df_stability, colors):
    """
    Rysuje wykres Stability: Spearman i Sem. Overlap dla Standard IG vs SmoothGrad.

    Argumenty:
        ax: Matplotlib axes do rysowania
        df_stability: DataFrame z Czesci 2
        colors: dict z paleta kolorow
    """
    groups = []
    std_ig_vals = []
    smooth_vals = []

    for para_type, para_label in [('mistral', 'Mistral'), ('synonym', 'Synonym')]:
        df_sub = df_stability[df_stability['para_type'] == para_type]
        if df_sub.empty:
            continue
        groups.append(f'Spearman\n({para_label})')
        std_ig_vals.append(df_sub['spearman_standard_ig'].mean())
        smooth_vals.append(df_sub['spearman_smoothgrad'].mean())

        groups.append(f'Sem.Overlap\n({para_label})')
        std_ig_vals.append(df_sub['sem_overlap_standard_ig'].mean())
        smooth_vals.append(df_sub['sem_overlap_smoothgrad'].mean())

    if not groups:
        ax.text(0.5, 0.5, 'Brak danych', ha='center', va='center', fontsize=12)
        return

    x = np.arange(len(groups))
    width = 0.35

    ax.bar(x - width / 2, std_ig_vals, width, label='Standard IG',
           color=colors['red'], alpha=0.85)
    ax.bar(x + width / 2, smooth_vals, width, label='SmoothGrad',
           color=colors['light_blue'])

    ax.set_ylabel('Wartosc metryki')
    ax.set_xticks(x)
    ax.set_xticklabels(groups, fontsize=8)
    ax.legend()
    ax.set_title('(b) Stability: Std IG vs SmoothGrad')
    ax.axhline(y=0, color='gray', linestyle='-', alpha=0.3)


def _plot_steering(ax, df_steering, colors):
    """
    Rysuje wykres Steering: Success Rate i Side Effect Rate per alpha.

    Argumenty:
        ax: Matplotlib axes do rysowania
        df_steering: DataFrame z Czesci 3
        colors: dict z paleta kolorow
    """
    x = np.arange(len(df_steering))
    alpha_labels = [f'{a:.0f}' for a in df_steering['alpha']]
    width = 0.35

    bars1 = ax.bar(x - width / 2, df_steering['success_rate'], width,
                   label='Success Rate', color=colors['blue'])
    bars2 = ax.bar(x + width / 2, df_steering['side_effect_rate'], width,
                   label='Side Effect Rate', color=colors['red'])

    ax.set_ylabel('Procent (%)')
    ax.set_xlabel('Alpha')
    ax.set_xticks(x)
    ax.set_xticklabels(alpha_labels)
    ax.legend()
    ax.set_title('(c) Steering: Skutecznosc detoksykacji')
    ax.axhline(y=80, color=colors['blue'], linestyle='--', alpha=0.4)
    ax.axhline(y=5, color=colors['red'], linestyle='--', alpha=0.4)
    ax.set_ylim(0, 105)

    # Etykiety na slupkach
    for bar in bars1:
        h = bar.get_height()
        ax.annotate(f'{h:.1f}%', xy=(bar.get_x() + bar.get_width() / 2, h),
                    xytext=(0, 3), textcoords='offset points', ha='center', fontsize=8)
    for bar in bars2:
        h = bar.get_height()
        ax.annotate(f'{h:.1f}%', xy=(bar.get_x() + bar.get_width() / 2, h),
                    xytext=(0, 3), textcoords='offset points', ha='center', fontsize=8)


def _generate_conclusions(df_fidelity, df_stability, df_steering):
    """
    Generuje automatyczne wnioski tekstowe na podstawie wynikow eksperymentalnych.

    Porownuje metody XAI (IG vs IxG), stabilnosc (Standard IG vs SmoothGrad)
    oraz skutecznosc steeringu (Success Rate vs Side Effects).

    Argumenty:
        df_fidelity: DataFrame z Czesci 1
        df_stability: DataFrame z Czesci 2
        df_steering: DataFrame z Czesci 3
    """
    print("\n" + "=" * 70)
    print("WNIOSKI Z EKSPERYMENTU")
    print("=" * 70)

    # --- Wniosek 1: Fidelity ---
    print("\n--- 1. FIDELITY (Wiernosc wyjasnien) ---")
    ig_comp = df_fidelity['ig_comprehensiveness'].mean()
    ixg_comp = df_fidelity['ixg_comprehensiveness'].mean()
    ig_suff = df_fidelity['ig_sufficiency'].mean()
    ixg_suff = df_fidelity['ixg_sufficiency'].mean()

    comp_winner = 'IG' if ig_comp > ixg_comp else 'IxG'
    comp_diff = abs(ig_comp - ixg_comp)
    print(f"  Comprehensiveness: {comp_winner} jest lepsza o {comp_diff:.4f}")
    print(f"    IG={ig_comp:.4f}, IxG={ixg_comp:.4f}")
    print(f"    (wyzsza wartosc = metoda lepiej identyfikuje kluczowe tokeny)")

    suff_winner = 'IG' if ig_suff < ixg_suff else 'IxG'
    suff_diff = abs(ig_suff - ixg_suff)
    print(f"  Sufficiency: {suff_winner} jest lepsza o {suff_diff:.4f}")
    print(f"    IG={ig_suff:.4f}, IxG={ixg_suff:.4f}")
    print(f"    (nizsza wartosc = same top tokeny wystarczaja do utrzymania predykcji)")

    # --- Wniosek 2: Stability ---
    print("\n--- 2. STABILITY (Stabilnosc wyjasnien) ---")
    for para_type, para_label in [('mistral', 'Mistral'), ('synonym', 'Synonimy')]:
        df_sub = df_stability[df_stability['para_type'] == para_type]
        if df_sub.empty:
            print(f"  {para_label}: Brak danych.")
            continue

        sp_std = df_sub['spearman_standard_ig'].mean()
        sp_smooth = df_sub['spearman_smoothgrad'].mean()
        sp_improvement = sp_smooth - sp_std
        sp_pct = (sp_improvement / abs(sp_std) * 100) if sp_std != 0 else 0

        so_std = df_sub['sem_overlap_standard_ig'].mean()
        so_smooth = df_sub['sem_overlap_smoothgrad'].mean()
        so_improvement = so_smooth - so_std
        so_pct = (so_improvement / abs(so_std) * 100) if so_std != 0 else 0

        print(f"  Parafrazy {para_label} (n={len(df_sub)}):")
        print(f"    Spearman: Std IG={sp_std:.4f}, SmoothGrad={sp_smooth:.4f} "
              f"(poprawa: {sp_improvement:+.4f}, {sp_pct:+.1f}%)")
        print(f"    Sem.Overlap: Std IG={so_std:.4f}, SmoothGrad={so_smooth:.4f} "
              f"(poprawa: {so_improvement:+.4f}, {so_pct:+.1f}%)")

    # --- Wniosek 3: Steering ---
    print("\n--- 3. STEERING (Sterowanie reprezentacjami) ---")
    best_idx = df_steering['success_rate'].idxmax()
    best = df_steering.loc[best_idx]
    optimal = best['success_rate'] > 80 and best['side_effect_rate'] < 5

    print(f"  Najlepsza wartosc alpha: {best['alpha']:.1f}")
    print(f"    Success Rate:     {best['success_rate']:.2f}% {'(cel >80% SPELNIONY)' if best['success_rate'] > 80 else '(cel >80% NIESPELNIONY)'}")
    print(f"    Side Effect Rate: {best['side_effect_rate']:.2f}% {'(cel <5% SPELNIONY)' if best['side_effect_rate'] < 5 else '(cel <5% NIESPELNIONY)'}")
    print(f"    Avg Delta Toxic:  {best['avg_delta_toxic']:+.4f}")
    print(f"    Avg Delta Safe:   {best['avg_delta_safe']:+.4f}")

    if optimal:
        print(f"\n  KONKLUZJA: Steering z alpha={best['alpha']:.1f} skutecznie detoksykuje model")
        print(f"  bez retrenowania, przy minimalnych efektach ubocznych.")
    else:
        print(f"\n  KONKLUZJA: Steering wymaga dalszego dostrojenia parametru alpha.")
        print(f"  Zaden z testowanych wariantow nie spelnia jednoczesnie obu kryteriow.")

    # --- Podsumowanie globalne ---
    print("\n" + "=" * 70)
    print("PODSUMOWANIE GLOBALNE")
    print("=" * 70)
    print(f"  1. Najlepsza metoda XAI pod wzgledem fidelity: {comp_winner} (Comprehensiveness)")
    print(f"  2. SmoothGrad poprawia stabilnosc wyjasnien w porownaniu do Standard IG")
    print(f"  3. Steering {'skutecznie' if optimal else 'czesciowo'} naprawia model bez retrenowania")


def generate_summary_report(df_fidelity, df_stability, df_steering, df_steering_details):
    """
    Generuje kompletny raport podsumowujacy z wykresami w stylu akademickim.

    Tworzy:
        - Zbiorcza tabele metryk (CSV)
        - Wykres zbiorczy 3-w-1 (PNG, 300 DPI, bez suptitle dla LaTeX)
        - Osobne wykresy per modul (PNG, 300 DPI)
        - Automatyczne wnioski tekstowe

    Argumenty:
        df_fidelity: DataFrame z Czesci 1
        df_stability: DataFrame z Czesci 2
        df_steering: DataFrame z Czesci 3 (summary)
        df_steering_details: DataFrame z Czesci 3 (per-sample)

    Zwraca:
        DataFrame z tabela zbiorczych metryk
    """
    print("\n>>> [CZESC 4] Generowanie raportu podsumowujacego...")

    # Styl akademicki
    colors = _setup_academic_style()

    # -----------------------------------------------
    # 4.1 TABELA ZBIORCZA METRYK
    # -----------------------------------------------
    print("    Budowanie zbiorczej tabeli metryk...")
    df_metrics = _build_metrics_table(df_fidelity, df_stability, df_steering)
    df_metrics.to_csv(f"{RESULTS_DIR}/summary_all_metrics.csv", index=False)

    print("\n=== ZBIORCZA TABELA METRYK ===")
    print(df_metrics[['Module', 'Metric', 'Value']].to_string(index=False))
    print(f"\n    Tabela zapisana do: {RESULTS_DIR}/summary_all_metrics.csv")

    # -----------------------------------------------
    # 4.2 WYKRES ZBIORCZY 3-W-1
    # -----------------------------------------------
    print("\n    Generowanie wykresu zbiorczego...")
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    _plot_fidelity(axes[0], df_fidelity, colors)
    _plot_stability(axes[1], df_stability, colors)
    _plot_steering(axes[2], df_steering, colors)

    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/summary_overview.png")
    plt.show()
    print(f"    Wykres zbiorczy zapisany: {RESULTS_DIR}/summary_overview.png")

    # -----------------------------------------------
    # 4.3 OSOBNE WYKRESY PER MODUL
    # -----------------------------------------------
    print("\n    Generowanie osobnych wykresow...")

    # Fig Fidelity
    fig_f, ax_f = plt.subplots(figsize=(6, 4.5))
    _plot_fidelity(ax_f, df_fidelity, colors)
    ax_f.set_title('')  # Bez tytulu - LaTeX caption
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/fig_fidelity.png")
    plt.show()

    # Fig Stability
    fig_s, ax_s = plt.subplots(figsize=(7, 4.5))
    _plot_stability(ax_s, df_stability, colors)
    ax_s.set_title('')
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/fig_stability.png")
    plt.show()

    # Fig Steering
    fig_st, ax_st = plt.subplots(figsize=(6, 4.5))
    _plot_steering(ax_st, df_steering, colors)
    ax_st.set_title('')
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/fig_steering.png")
    plt.show()

    print(f"    Osobne wykresy zapisane: fig_fidelity.png, fig_stability.png, fig_steering.png")

    # -----------------------------------------------
    # 4.4 AUTOMATYCZNE WNIOSKI
    # -----------------------------------------------
    _generate_conclusions(df_fidelity, df_stability, df_steering)

    print("\nCzesc 4 zakonczona.")
    return df_metrics

In [ ]:
df_metrics = generate_summary_report(df_fidelity, df_stability, df_steering, df_steering_details)

In [ ]:
# ===================================================
# 4.5 LISTA WYGENEROWANYCH PLIKOW
# ===================================================


def list_generated_files():
    """
    Wyswietla liste wszystkich plikow wygenerowanych przez notebook
    w katalogu RESULTS_DIR z informacja o rozmiarze i typie.
    """
    print("\n" + "=" * 70)
    print("WYGENEROWANE PLIKI")
    print("=" * 70)

    type_map = {
        '.csv': 'CSV',
        '.png': 'PNG',
        '.npy': 'NumPy',
        '.txt': 'Text',
    }

    total_size = 0
    file_count = 0

    files = sorted(os.listdir(RESULTS_DIR))
    for f in files:
        file_path = os.path.join(RESULTS_DIR, f)
        if os.path.isfile(file_path):
            size_kb = os.path.getsize(file_path) / 1024
            total_size += size_kb
            file_count += 1
            ext = os.path.splitext(f)[1].lower()
            ftype = type_map.get(ext, ext)
            print(f"  {f:40s} {ftype:6s} {size_kb:8.2f} KB")

    print(f"\n  {'RAZEM':40s} {'':6s} {total_size:8.2f} KB ({file_count} plikow)")
    print(f"\n  Katalog wynikow: {RESULTS_DIR}")

In [ ]:
list_generated_files()